# McGreasy Diet Problem

A simple diet model

In [1]:
import sys
import gamspy as gp

In [2]:
options = gp.Options(equation_listing_limit=100)
m = gp.Container(load_from='mcgreasy.gdx',options=options)
# extract the sets and parameters for the gdx file
food, nutr, a, min_nutr, cost = m.getSymbols(["food","nutr","a","min_nutr","cost"])
for name, symbol in m:
    print(f"Name: {name}, records: {symbol.records}")

Name: food, records:   uni element_text
0  QP             
1  MD             
2  BM             
3  FF             
4  MC             
5  FR             
6  SM             
7  1M             
8  OJ             
Name: nutr, records:     uni element_text
0  Prot             
1  VitA             
2  VitC             
3  Calc             
4  Iron             
5  Cals             
6  Carb             
Name: a, records:     nutr food  value
0   Prot   QP   28.0
1   Prot   MD   24.0
2   Prot   BM   25.0
3   Prot   FF   14.0
4   Prot   MC   31.0
5   Prot   FR    3.0
6   Prot   SM   15.0
7   Prot   1M    9.0
8   Prot   OJ    1.0
9   VitA   QP   15.0
10  VitA   MD   15.0
11  VitA   BM    6.0
12  VitA   FF    2.0
13  VitA   MC    8.0
14  VitA   SM    4.0
15  VitA   1M   10.0
16  VitA   OJ    2.0
17  VitC   QP    6.0
18  VitC   MD   10.0
19  VitC   BM    2.0
20  VitC   MC   15.0
21  VitC   FR   15.0
22  VitC   1M    4.0
23  VitC   OJ  120.0
24  Calc   QP   30.0
25  Calc   MD   20.0
26  Calc   BM  

In [3]:
x = gp.Variable(m,'x','positive',domain=[food],description="Number of each type of food to eat")

min_nutr_eqn = gp.Equation(m,'min_nutr_eqn',domain=[nutr],description="Minimum Daily Requirement")
min_nutr_eqn[nutr]= (
    gp.Sum(food, a[nutr,food]*x[food]) >= min_nutr[nutr] )

diet = gp.Model(m,'diet',
                   equations = [min_nutr_eqn],
                   problem = "LP",
                   sense = "MIN",
                   objective = gp.Sum(food, cost[food]*x[food]),
                  )

diet.solve()

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,14.855738,8,10,LP,CPLEX,0.0


In [4]:
# view all the foods that were eaten
xvals = x.records[['food','level']].set_index('food').copy()
display(xvals[xvals['level']!=0])

,level
food,
QP,4.385246
FR,6.147541
1M,3.422131


## Change the problem

What if this person really liked beef?

In [ ]:
# new set of food with beef
beef = gp.Set(m,'beef',domain=[food],records=['QP', 'MD', 'BM'])

# we want to maxsize the beef intake
beef1 = gp.Model(m,'beef1',
                   equations = [min_nutr_eqn],
                   problem = "LP",
                   sense = "MAX",
                   objective = gp.Sum(beef,x[beef])
                  )

beef1.solve(solver="highs")
# notice how the model is unbounded, none of teh vars have upper lims

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,Unbounded,10.606061,8,10,LP,HIGHS,0.016


In [6]:
# these are the nutrients we'll set limits on
bad_nutr = gp.Set(m,'bad_nutr',domain=[nutr],records=['Cals', 'Carb'])
max_nutr = gp.Parameter(m,'max_nutr',domain=[nutr])
max_nutr[bad_nutr] = 2 * min_nutr[bad_nutr]

max_nutr_req = gp.Equation(m,'max_nutr_req',domain=[nutr])
max_nutr_req[bad_nutr]=(
    gp.Sum(food,a[bad_nutr,food]*x[food]) <= max_nutr[bad_nutr] )

beef1 = gp.Model(m,'beef1',
                   equations = [min_nutr_eqn,max_nutr_req],
                   problem = "LP",
                   sense = "MAX",
                   objective = gp.Sum(beef,x[beef])
                  )

beef1.solve()

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,10.810811,10,10,LP,CPLEX,0.015


In [8]:
xvals = x.records[['food','level']].set_index('food').copy()
display(xvals[xvals['level']!=0])

,level
food,
MD,10.810811


In [10]:
# deactivating the calorie constraint
bad_nutr['Cals'] = False
beef1.solve()

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,21.212121,9,10,LP,CPLEX,0.0


In [12]:
bad_nutr['Cals'] = True
options.equation_listing_limit=100

# want to create a more 'normal diet'
sandwich_eqn = gp.Equation(m,'sandwich_eqn',description="Only 3 sandwiches per day")
sandwich_eqn[:]= x["QP"] + x["MD"] + x["BM"] + x["FF"] + x["MC"] + x["SM"] <= 3

drinking_eqn = gp.Equation(m,'drinking_eqn',description="Only 3 drinks per day")
drinking_eqn[:]= x["OJ"] + x["1M"] <= 3

norm = gp.Model(m,'norm',
                   equations = [min_nutr_eqn, max_nutr_req, sandwich_eqn, drinking_eqn],
                   problem = "LP",
                   sense = "MAX",
                   objective = gp.Sum(beef,x[beef])
                  )

# Limit french fries too
x.up['FR'] = 2

norm.solve()

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,InfeasibleGlobal,-182.0,12,10,LP,CPLEX,0.0


In [13]:
# we are a little too strict with the constraints leading to "Infeasibility"

# using a slack variable, can be negative if problem calls for it
s = gp.Variable(m,'s','free',description="Surplus variable for drinking_eqn")
drinking_eqn[:]= x["OJ"] + x["1M"] <= 3 + s

mindrink = gp.Model(m,'mindrink',
                   equations = [min_nutr_eqn, max_nutr_req, sandwich_eqn, drinking_eqn],
                   problem = "LP",
                   sense = "MIN",
                   objective = s
                  )

s.lo[:] = 0

mindrink.solve()

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,16.9,11,10,LP,CPLEX,0.0


In [ ]:
xvals = x.records[['food','level']].set_index('food').copy()
display(xvals[xvals['level']!=0])
# that's a crazy amount of juice!

,level
food,
QP,3.0
FR,2.0
1M,1.9
OJ,18.0


### Going to do some analysis on specific vitamins

In [16]:
jtlvitc = gp.Parameter(m,'jtlvitc')
jtlvitc[:] = gp.Sum(food, a["VitC",food] * x.l[food])
display(jtlvitc.records,jtlvitc.toValue())

,value
0,2215.6


np.float64(2215.6)

In [18]:
# hold the amount of extra drink that we just found
options.hold_fixed_variables = False
s.fx[:] = s.l

# Note that norm model will be regenerated with new drinking_eqn
norm.solve()
display(s.records,s.toValue(),x.records)

,level,marginal,lower,upper,scale
0,16.9,-0.0,16.9,16.9,1.0


np.float64(16.9)

,food,level,marginal,lower,upper,scale
0,QP,3.0,0.0,0.0,inf,1.0
1,MD,0.0,-0.0,0.0,inf,1.0
2,BM,0.0,-0.0,0.0,inf,1.0
3,FF,0.0,-1.0,0.0,inf,1.0
4,MC,0.0,-1.0,0.0,inf,1.0
5,FR,2.0,-0.0,0.0,2.0,1.0
6,SM,0.0,-1.0,0.0,inf,1.0
7,1M,1.9,0.0,0.0,inf,1.0
8,OJ,18.0,0.0,0.0,inf,1.0


## Now we're going to be dealing with proportional constraints

In [ ]:
cals_eaten = gp.Variable(m,"cals_eaten","positive",description="Number of calories ingested")
cals_eaten.up[:] = 10000

cals_eaten_eqn = gp.Equation(m,'cals_eaten_eqn')
cals_eaten_eqn[:]= cals_eaten == gp.Sum(food,a["Cals",food] * x[food])

vitc_eaten = gp.Variable(m,"vitc_eaten","positive",description="Amount of vitamin C I eat")
vita_eaten = gp.Variable(m,"vita_eaten","positive",description="Amount of vitamin A I eat")

vitc_eaten_eqn = gp.Equation(m,'vitc_eaten_eqn')
vitc_eaten_eqn[:]= vitc_eaten == gp.Sum(food,a["VitC",food] * x[food])

vita_eaten_eqn = gp.Equation(m,'vita_eaten_eqn')
vita_eaten_eqn[:]= vita_eaten == gp.Sum(food,a["VitA",food] * x[food])

limit_vitc_ratio_eqn = gp.Equation(m,'limit_vitc_ratio_eqn')
limit_vitc_ratio_eqn[:]= cals_eaten / vitc_eaten <= 40 

limit_vita_ratio_eqn = gp.Equation(m,'limit_vita_ratio_eqn')
limit_vita_ratio_eqn[:]= cals_eaten / vita_eaten <= 40 

ratiomodel = gp.Model(m,'ratiomodel',
                   equations = [min_nutr_eqn,cals_eaten_eqn,vitc_eaten_eqn,vita_eaten_eqn,limit_vitc_ratio_eqn,limit_vita_ratio_eqn],
                   problem = "LP",
                   sense = "MAX",
                   objective = gp.Sum(beef,x[beef])
                  )

ratiomodel.solve()
# this isn't solvable because it's non linear! (division is not good in linear programming)

GamspyException: Return code 2. There was a compilation error. Check C:\Users\plang\AppData\Local\Temp\tmpi11ulbcz\_vNoO48Z2THucwO1qXav3kA.lst for more information.

=============
Error Summary
=============
1098  Parameter autogenerated_domUsd_mTvYeQEpCR22Qg_WevqOlXw / /;
****          $55,256
**** LINE      8 INPUT       C:\Users\plang\AppData\Local\Temp\tmpi11ulbcz\_vNoO48Z2THucwO1qXav3kA.gms
****  55  Endogenous operands for / not allowed in linear models
**** 256  Error(s) in analyzing solve statement.
**** The following LP errors were detected in model ratiomodel:
****  55 equation limit_vitc_ratio_eqn.. VAR operands for /
****  55 equation limit_vita_ratio_eqn.. VAR operands for /

**** 2 ERROR(S)   0 WARNING(S)


COMPILATION TIME     =        0.016 SECONDS      4 MB  50.4.1 84b10359 WEX-WEI


USER: Academic User                                  G250906+0003Ac-GEN
      rinkam@wisc.edu                                         GPA107788
      License for teaching and research at d

In [22]:
# switch to non-linear solver, but also fix bounds to avoid dividing by 0
vitc_eaten.l[:] = 1.0 
vita_eaten.l[:] = 1.0

ratiomodel = gp.Model(m,'ratiomodel',
                   equations = [min_nutr_eqn,cals_eaten_eqn,vitc_eaten_eqn,vita_eaten_eqn,limit_vitc_ratio_eqn,limit_vita_ratio_eqn],
                   problem = "NLP",
                   sense = "MAX",
                   objective = gp.Sum(beef,x[beef])
                  )

ratiomodel.solve()

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalLocal,27.027027,13,13,NLP,IPOPTH,0.026


In [ ]:
# keeping the solution linear 

# just do some simple algebra
linear_limit_vitc_ratio_eqn = gp.Equation(m,'linear_limit_vitc_ratio_eqn')
linear_limit_vitc_ratio_eqn[:]= cals_eaten <= 40 * vitc_eaten

linear_limit_vita_ratio_eqn = gp.Equation(m,'linear_limit_vita_ratio_eqn')
linear_limit_vita_ratio_eqn[:]= cals_eaten <= 40 * vita_eaten  

linear_ratiomodel = gp.Model(m,'ratiomodel',
                   equations = [min_nutr_eqn,cals_eaten_eqn,vitc_eaten_eqn,vita_eaten_eqn,linear_limit_vitc_ratio_eqn,linear_limit_vita_ratio_eqn],
                   problem = "LP",
                   sense = "MAX",
                   objective = gp.Sum(beef,x[beef])
                  )

linear_ratiomodel.solve()
# got the same exact answer

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,27.027027,13,13,LP,CPLEX,0.0


In [24]:
xvals = x.records[['food','level']].set_index('food').copy()
display(xvals[xvals['level']!=0])

,level
food,
MD,27.027027
